In [18]:
import numpy as np
import pandas as pd

from PIL import Image

import matplotlib.pyplot as plt


import generate_image_labels as giLabels
import edge_feature_generation as efg

%matplotlib inline
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14,10)

In [43]:
full_df = giLabels.gen_retrain_labels()

In [7]:
full_df.head()

,img_path,label,height_brick,height_plate,height_other,shape_corner,shape_round,shape_square,short_1,short_2,...,short_6,short_8,long_1,long_2,long_3,long_4,long_6,long_8,long_10,long_12
0,/home/sean/Desktop/lego_dirty_close/brick_corn...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,/home/sean/Desktop/lego_dirty_close/brick_corn...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,/home/sean/Desktop/lego_dirty_close/brick_corn...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,/home/sean/Desktop/lego_dirty_close/brick_corn...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,/home/sean/Desktop/lego_dirty_close/brick_corn...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [35]:
plt.rcParams['figure.figsize'] = (7,5)

In [85]:
reload(efg)

<module 'edge_feature_generation' from 'edge_feature_generation.py'>

In [89]:
def flip( inp_arr ):
    return inp_arr[::-1]

In [110]:
# Go image by image
# pass through image processing pipeline to generate features
# Flip, rotate, each time rot/flipping rc sums and rc ratio
# Dont actually need to rotate or process each image

out_df = pd.DataFrame( 
                        columns=['img_path']+
                                ['rc_ratio']+
                                ['row_'+str(i) for i in range(0,126)]+
                                ['col_'+str(i) for i in range(0,126)]
                     )

img_counter = 0
for i in range( 0, full_df.shape[0], 2000 ):

    inp_img = full_df.loc[i,'img_path']
    edge_arr = efg.get_img_edge_data( 
                                inp_img,
                                size=[128,128],
                                edge_cutoff=20,
                                edge_smooth=3,
                                rot_img=False,
                                display=False,
                                ret_arr=True,
                               )

    
    rc_ratio, row, col = efg.get_edge_features( edge_arr, display=False )
    
#    row, col
    row_col_combinations = [
                                [       row  ,       col  ],
                                [       row  , flip( col )],
                                [ flip( row ),       col  ],
                                [ flip( row ), flip( col )],       
                                [       col  ,       row  ],
                                [       col  , flip( row )],
                                [ flip( col ),       row  ], 
                                [ flip( col ), flip( row )],
                           ]

    for j in range( 0, len(row_col_combinations) ):
    
        rc = rc_ratio
        if ( j > 3 ):
            rc = 1./rc_ratio

        out_list = [inp_img,rc,]+row_col_combinations[j][0].tolist()+row_col_combinations[j][1].tolist()
        
        out_df.loc[img_counter,:] = out_list
    
        img_counter = img_counter + 1

In [112]:
out_df

,img_path,rc_ratio,row_0,row_1,row_2,row_3,row_4,row_5,row_6,row_7,...,col_116,col_117,col_118,col_119,col_120,col_121,col_122,col_123,col_124,col_125
0,/home/sean/Desktop/lego_dirty_close/brick_corn...,0.992063,0.461218,0.62201,0.0451345,1.31877,0.784212,1.20735,1.1876,1.33429,...,1.12836,1.31172,1.57548,1.17914,1.18619,0.984496,1.26941,0.0169254,0.600853,0.708047
1,/home/sean/Desktop/lego_dirty_close/brick_corn...,0.992063,0.461218,0.62201,0.0451345,1.31877,0.784212,1.20735,1.1876,1.33429,...,1.17632,1.23274,1.35121,1.43584,1.50636,1.43443,1.24966,0.0141045,0.469681,0.55995
2,/home/sean/Desktop/lego_dirty_close/brick_corn...,0.992063,0.633293,0.447114,0.126941,1.18619,1.00283,1.13824,1.13683,1.07194,...,1.12836,1.31172,1.57548,1.17914,1.18619,0.984496,1.26941,0.0169254,0.600853,0.708047
3,/home/sean/Desktop/lego_dirty_close/brick_corn...,0.992063,0.633293,0.447114,0.126941,1.18619,1.00283,1.13824,1.13683,1.07194,...,1.17632,1.23274,1.35121,1.43584,1.50636,1.43443,1.24966,0.0141045,0.469681,0.55995
4,/home/sean/Desktop/lego_dirty_close/brick_corn...,1.008,0.55995,0.469681,0.0141045,1.24966,1.43443,1.50636,1.43584,1.35121,...,1.02117,1.14952,1.07194,1.13683,1.13824,1.00283,1.18619,0.126941,0.447114,0.633293
5,/home/sean/Desktop/lego_dirty_close/brick_corn...,1.008,0.55995,0.469681,0.0141045,1.24966,1.43443,1.50636,1.43584,1.35121,...,1.40904,1.00283,1.33429,1.1876,1.20735,0.784212,1.31877,0.0451345,0.62201,0.461218
6,/home/sean/Desktop/lego_dirty_close/brick_corn...,1.008,0.708047,0.600853,0.0169254,1.26941,0.984496,1.18619,1.17914,1.57548,...,1.02117,1.14952,1.07194,1.13683,1.13824,1.00283,1.18619,0.126941,0.447114,0.633293
7,/home/sean/Desktop/lego_dirty_close/brick_corn...,1.008,0.708047,0.600853,0.0169254,1.26941,0.984496,1.18619,1.17914,1.57548,...,1.40904,1.00283,1.33429,1.1876,1.20735,0.784212,1.31877,0.0451345,0.62201,0.461218
8,/home/sean/Desktop/lego_dirty_close/brick_1x8/...,1,1.45505,1.49297,0.954234,0.796248,0.862603,0.791509,0.698297,0.657221,...,0.767811,0.647742,0.767811,0.804148,0.849964,0.712516,0.854703,0.968453,1.74574,1.5609
9,/home/sean/Desktop/lego_dirty_close/brick_1x8/...,1,1.45505,1.49297,0.954234,0.796248,0.862603,0.791509,0.698297,0.657221,...,0.737794,0.747273,0.606665,0.870502,0.788349,0.744113,0.796248,0.908418,1.68097,1.63989
